In [ ]:
import numpy as np
import pandas as pd
import altair as alt
from vega_datasets import data
import requests
import time
import geopandas
from gpx_converter import Converter
from shapely.geometry import LineString
import matplotlib.pyplot as plt
from os.path import exists as file_exists
import warnings
warnings.filterwarnings(action='once')

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler 
from sklearn import metrics, tree

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

from spacy.lang.en.stop_words import STOP_WORDS
STOP_WORDS = STOP_WORDS.union({'ll', 've', 'pron'})

In [ ]:
# calculates the sinuosity of each route from its gpx file of lat/lon coordinates
def calcluate_sinuosity(gpx_file_num):
    gpx_file = f'gpx_files/{str(gpx_file_num)}.gpx'
    if file_exists(gpx_file):
        try:
            gpx_array = Converter(input_file=gpx_file).gpx_to_numpy_array()
        except Exception:
            return -1
        
        splits = 4
        subsets = np.array_split(gpx_array, splits)
        subset_sinuosities = []
        
        for subset in subsets:
            start_pt = subset[0]
            end_pt = subset[-1]
            route = LineString(subset)
            route_SL = LineString((start_pt, end_pt))
            route_sinuosity = route.length / route_SL.length
            subset_sinuosities.append(route_sinuosity)
        return sum(subset_sinuosities)/splits
    else:
        return -2

# Data input & cleaning

In [ ]:
route_data_RAW = pd.read_csv('route_data_RAW.csv')
s = [calcluate_sinuosity(x) for x in route_data_RAW['gpx_file_num']]
route_data_RAW['sinuosity'] = s
#route_data_RAW.info()

We only want routes that have ratings in our training set.

In [ ]:
comments = (
    pd.read_csv('comments.csv')
    .drop('files', axis=1)
    .dropna()
    .groupby('route_name', as_index=False)
    .agg(lambda x: ' '.join(x))
    .drop_duplicates()
)

rated_roads = (
    route_data_RAW.query('num_user_reviews > 0 and sinuosity >= 0')
    #.merge(comments, how='left', left_on='name',right_on='route_name')
    .fillna(' ')
)

And we will limit scope to routes in the US, including routes that cross state lines.

In [ ]:
valid_states = ['Alabama', 'California', 'Georgia', 'Missouri', 'Illinois', 'Ohio',
       'Kentucky', 'Colorado', 'United States', 'Indiana', 'New York',
       'Vermont', 'Texas', 'Florida', 'Minnesota', 'Virginia',
       'Oklahoma', 'Arkansas', 'Maryland', 'West Virginia',
       'Michigan', 'North Carolina', 'Oregon', 'Pennsylvania',
       'Washington', 'New Jersey', 'Alaska',
       'South Carolina', 'Utah', 'New Hampshire', 'Iowa', 'Louisiana',
       'Mississippi', 'Wisconsin',
       'South Dakota', 'Wyoming', 'Massachusetts', 'New Mexico',
       'Montana', 'Idaho', 'Nevada', 'Arizona',
       'Kansas', 'Northeast', 'Southwest', 'Golf Coast', 'Southeast',
       'Tennessee', 'Nebraska', 'Delaware', 'Pacific Coast',
       'Appalachian Mountains', 'Maine', 'Rhode Island', 'Connecticut',
       'North Dakota', 'Hawaii']
us_route_data = rated_roads[rated_roads.state.isin(valid_states)]

In [ ]:
#us_route_data['weighted_rating'] = us_route_data['user_rating'] * us_route_data['num_user_reviews']
us_route_data['description'] = us_route_data.agg(lambda x: f"{x['scenery_description']}, {x['drive_enjoyment_description']}, {x['tourism_description']}", axis=1)

In [ ]:
us_route_data['description'][0]

In [ ]:
# Make a geoDataFrame with route coords as the geometry
def get_route_coords(gpx_file_num):
    gpx_file = f'gpx_files/{str(gpx_file_num)}.gpx'
    if file_exists(gpx_file):
        try:
            gpx_df = Converter(input_file=gpx_file).gpx_to_dataframe()
            route_line = LineString(list(zip(gpx_df.longitude, gpx_df.latitude)))
            return route_line
        except Exception:
            return None

route_coords = {
    'gpx_file_num': [x for x in us_route_data['gpx_file_num']],
    'geometry': [get_route_coords(x) for x in us_route_data['gpx_file_num']]
}

#also used EPSG: 4326
route_coords_gdf = geopandas.GeoDataFrame(route_coords, crs="EPSG:4269").merge(us_route_data, on='gpx_file_num')

Dataset of NPS Park boundaries: https://irma.nps.gov/DataStore/Reference/Profile/2225713

In [ ]:
# # # calculate a route's shortest distance to a NPS site 
# park_data = geopandas.read_file("nps_boundary/nps_boundary.shp")

# route_coords_gdf['centroid'] = route_coords_gdf['geometry'].to_crs(epsg=4269).centroid
# park_data['centroid'] = park_data['geometry'].to_crs(epsg=4269).centroid

# us_route_data['distance2nps'] = route_coords_gdf.apply(lambda x: park_data['centroid'].distance(x['centroid']).min(),axis=1)

In [ ]:
#Final dataset for analyses
route_coords_gdf.head()

# Visualizations

In [ ]:
route_coords_gdf.head()

In [ ]:
rode = alt.Chart(route_coords_gdf).mark_point().encode(
    x='user_rating',
    y='num_users_rode',
    tooltip=['name','state','user_rating']
).interactive()

want2ride = alt.Chart(route_coords_gdf).mark_point().encode(
    x='user_rating',
    y='num_users_want2ride',
    tooltip=['name','state','user_rating']
).interactive()

rode | want2ride

In [ ]:
scenery_chart = alt.Chart(rated_roads).mark_circle().encode(
    alt.X('scenery_rating', bin=True),
    alt.Y('user_rating', bin=True),
    size='count()'
)

drive_enjoyment_chart = alt.Chart(rated_roads).mark_circle().encode(
    alt.X('drive_enjoyment_rating', bin=True),
    alt.Y('user_rating', bin=True),
    size='count()'
)

tourism_chart = alt.Chart(rated_roads).mark_circle().encode(
    alt.X('tourism_rating', bin=True),
    alt.Y('user_rating', bin=True),
    size='count()'
)

scenery_chart | drive_enjoyment_chart | tourism_chart

In [ ]:
alt.Chart(us_route_data).mark_point().encode(
    x='distance2nps',
    y='user_rating',
    tooltip=['name','state','user_rating']
).interactive()

## Geospatial features

In [ ]:
# US states background
states = alt.topo_feature(data.us_10m.url, feature='states')
background = alt.Chart(states).mark_geoshape(
    fill='lightgray',
    stroke='white'
).properties(
    width=1000,
    height=600
).project('albersUsa')

# routes
lines = alt.Chart(route_coords_gdf).mark_geoshape(
    filled=False,
    strokeWidth=1
).encode(color='user_rating')

background + lines

# Analyses

## Numerical features & Bag of words

In [ ]:
route_coords_gdf.info()

In [ ]:
# load/split data
X = route_coords_gdf
y = route_coords_gdf['weighted_rating']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=17)

# select features
numeric_features = ['sinuosity']#,'distance2nps']
text_features = ['scenery_description','drive_enjoyment_description','tourism_description']

# preprocessing
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="mean")), ("scaler", RobustScaler())]
)
text_transformer = TfidfVectorizer(stop_words=STOP_WORDS.union({'10'}), 
                                   ngram_range=(1,2),
                                   min_df=.05
                                  )

preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_features),
    #('comments',text_transformer, 'comments'),
#     ('scenery', text_transformer, 'scenery_description'),
#     ('drive_enjoyment', text_transformer, 'drive_enjoyment_description'),
#     ('tourism', text_transformer, 'tourism_description')
])


# pipeline
est = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RidgeCV())
])

est.fit(X_train,y_train)
y_pred = est.predict(X_train)
print("Mean absolute error:", metrics.mean_absolute_error(y_train, y_pred))
print("Mean squared error:", metrics.mean_squared_error(y_train, y_pred))
print("R^2:", metrics.r2_score(y_train, y_pred))

## Sentiment Analysis

In [ ]:
# split routes into hi and low ratings
sorted_ratings = us_route_data.sort_values(by=['user_rating','num_user_reviews'],ascending=False).reset_index()
hi,mid,low = np.split(sorted_ratings,[int(.3*len(sorted_ratings)), int(.6*len(sorted_ratings))])

# add rating labels
hi['rating_label'] = 'hi'
low['rating_label'] = 'low'

# merge labeled data
polar_data = pd.concat([hi,low],ignore_index=True)
polar_X = polar_data['comments']
polar_y = polar_data['rating_label']

# split data
X_train, X_test, y_train, y_test = train_test_split(polar_X, polar_y, test_size=0.2, random_state=17)


polar_pipe = Pipeline([
    ('vectorizer', TfidfVectorizer(stop_words=STOP_WORDS.union({'10'}), 
                                   ngram_range=(1,2),
                                   min_df=.1, 
                                   max_features = 500)),
    ('classifier', MultinomialNB())
])

polar_pipe.fit(X_train,y_train)

In [ ]:
polar_pipe[0].get_feature_names_out()

In [ ]:
vocab = polar_pipe.get_params()['vectorizer'].vocabulary_ 
                                                  
coeff_pos = polar_pipe.get_params()['classifier'].feature_log_prob_[0] 
coeff_neg = polar_pipe.get_params()['classifier'].feature_log_prob_[1]


from numpy import argsort

polarity = coeff_pos - coeff_neg
indices = argsort(polarity) # indices of the polarity list, sorted from least to greatest


print("Top Words \n-----")
for word in vocab:
    if vocab[word] in indices[-25:]:
        print(word)
        
# print("\nNegative Words \n-----")
# for word in vocab:
#     if vocab[word] in indices[:25]:
#         print(word) 

In [ ]:
# Maps routes w/ parks

# US states background
states = alt.topo_feature(data.us_10m.url, feature='states')
background = alt.Chart(states).mark_geoshape(
    fill='lightgray',
    stroke='white'
).properties(
    width=1000,
    height=600
).project('albersUsa')

# MR routes
lines = alt.Chart(route_coords_gdf).mark_geoshape(
    filled=False,
    strokeWidth=1
).encode(color='user_rating')


# NPS sites
parks = alt.Chart(park_data).mark_geoshape(
        color='brown',
        filled=False,
        strokeWidth=1)


background + lines + parks

In [ ]:
# topic modeling on comments to generate 'category' features for each route
# recommendation engine for routes

# Topic-modeling route comments

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
#import data
comments = pd.read_csv('comments.csv')[['comments']].dropna().drop_duplicates()
comments['comments']

### Preprocessing

In [ ]:
import spacy
import re
from spacy.lang.en.stop_words import STOP_WORDS

nlp = spacy.load('en_core_web_sm', disable=['parser','ner','tagger', 'textcat', 'lemmatizer'])

def my_lemmatizer(doc):
    doc_cleaned = ' '.join(re.findall(r'\b\w[\w\']+\b', doc))
    return [ w.lemma_.lower() for w in nlp(doc_cleaned) 
                      if w.lemma_ not in ['_', '.', '-PRON-'] ]

# stopwords = spacy.lang.en.stop_words.STOP_WORDS

# stopwords = set(my_lemmatizer(' '.join(list(stopwords))))

STOP_WORDS = STOP_WORDS.union({'ll', 've', 'pron',
                               'good','great','nice',
                               'ride', 'route','road'
                              })

### Clustering

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [ ]:
max_features = 100000

cv = CountVectorizer(#tokenizer=my_lemmatizer, 
                     stop_words=STOP_WORDS, 
                     ngram_range=(1, 3), 
                     min_df=0.1, 
                     max_df=0.95, 
                     #max_features=max_features
                    )
counts = cv.fit_transform(comments['comments'])

tf = TfidfTransformer()
matrix = tf.fit_transform(counts)

In [ ]:
range_of_clusters = range(2,11)

for n in range_of_clusters:
    kmc = KMeans(n_clusters=n, n_init=3, random_state=17)  # random_state for consistency
    kmc.fit(matrix)

    number_of_top_words = 10

    cluster_words = np.argsort(kmc.cluster_centers_, axis=1)
    terms = cv.get_feature_names_out()

    top_words = set()

    for i in range(n):
    #     print('Cluster {}: '.format(i))
    #     print(', '.join([terms[k] for k in cluster_words[i][-number_of_top_words:]]),'\n')
        top_words = top_words.union([terms[k] for k in cluster_words[i][-number_of_top_words:]])

    top_words = sorted(list(top_words))

    #combine word counts with cluster labels
    terms = cv.get_feature_names_out()
    word_df = pd.DataFrame(counts.toarray(), columns=terms)[top_words]
    word_df['Cluster'] = kmc.labels_.tolist()

    print(f'Clusters: {n}  Silhouette score: {silhouette_score(matrix, word_df.Cluster)}')

In [ ]:
#fit 
number_of_clusters = 5

kmc = KMeans(n_clusters=number_of_clusters, n_init=3, random_state=17)  # random_state for consistency
kmc.fit(matrix)

In [ ]:
number_of_top_words = 7

cluster_words = np.argsort(kmc.cluster_centers_, axis=1)
terms = cv.get_feature_names_out()

top_words = set()

for i in range(number_of_clusters):
    print('Cluster {}: '.format(i))
    print(', '.join([terms[k] for k in cluster_words[i][-number_of_top_words:]]),'\n')
    top_words = top_words.union([terms[k] for k in cluster_words[i][-number_of_top_words:]])
    
top_words = sorted(list(top_words))

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2, random_state=117)
matrix_pca = pca.fit_transform(matrix.toarray())

matrix_pca.shape

In [ ]:
import matplotlib.pyplot as plt

plt.style.use('seaborn')
plt.scatter(matrix_pca[:,0], matrix_pca[:,1], c=word_df['Cluster'], 
            cmap='viridis', alpha=0.15)
plt.gca().set(title='Plot for 2-Dimensional PCA Projection', 
              xlabel='PCA component 1', ylabel='PCA component 2');

In [ ]:
word_df.groupby('Cluster').count()[top_words[0]].\
    plot.bar(rot=0).\
    set(ylabel='Document count',
    title='Number of Documents per Cluster');

In [ ]:
word_df.groupby('Cluster').sum().transpose().\
    plot.bar(figsize=(13,5), width=0.7).\
    set(ylabel='Word frequency', 
    title='Word Frequencies by Topic, Combining the Top {} Words in Each Topic'.format(number_of_top_words));

# Extra Code

In [ ]:
# used to get gpx files from MotoRoads site

# for gpx in gpxs:
#     moto = requests.get('https://www.motorcycleroads.com/downloadgpx/' + str(gpx))
#     out = moto.text
#     name = str(gpx) + '.gpx'
#     with open(name, 'w') as f:
#         f.write(out)
#     time.sleep(2)